### Import necessary packages, define dxf parsing functions

In [6]:
import re
import sys
import glob
import struct
from scipy.optimize import curve_fit
from scipy.special import erf
from scipy.stats import linregress
from scipy.signal import medfilt
%pylab

def nullify(s):
    """Effectively converts UTF-8 encoded text into UTF-16"""
    s2 = "\0".join(list(s)) + "\0"
    return s2.encode()

def process_line_o2(b):
    """Returns one 'line' (data point) of oxygen data"""
    return struct.unpack("<fddd", b)

def process_line_n2(b):
    """Returns one 'line' (data point) of nitrogen data"""
    return struct.unpack("<fdd", b)

def get_odata(bites, pattern=b"CEvalGCData"):
    ostart = re.search(pattern, bites)
    nions = 3
    
    # nbytes is the total number of bytes containing time, 3 cups of data
    
    nbytes = struct.unpack("<I", bites[ostart.end() + 4:ostart.end() + 8])[0]
    no = nbytes / (4 + 8 * nions)
    odata = []
    for i in range(no):
        line = bites[ostart.end() + 28*i + 8: ostart.end() + 8 + 28*(i+1)]
        odata += [process_line_o2(line)]
    return array(odata)

def get_ndata(bites, pattern=b"CStringArray.+N\0002\0"):
    
    nstart = re.search(pattern, bites)
    nbytes = struct.unpack("<I", bites[nstart.end() + 22:nstart.end() + 26])[0]
    nions = 2
    nlines = nbytes / (4 + 8 * nions)
    ndata = []
    for i in range(nlines):
        line = bites[nstart.end() + 26 + i*20: nstart.end() + 26 + 20*(i+1)]
        ndata += [process_line_n2(line)]
    return array(ndata)

# Define the peakfinding function

def peakfind(data, riselim, decay_start, decay_end, offset):
    """Peak finding code that uses slopes of peaks for definition"""
    grad = gradient(data)
    downgrad = gradient(data, 5)
    l2 = []
    started = 0
    decaying = 0
    for i in range(len(grad)):
        if not started:
            if grad[i] > riselim:
                l2 += [i]
                started = 1
        elif started and grad[i] > riselim:
            continue
        elif grad[i] < decay_start and started and not decaying:
            decaying = 1
        elif started and decaying and downgrad[i] > decay_end:
            l2 += [i]
            started = 0
            decaying = 0
            return array(l2) + offset


def rtod(ratio, std):
    """Helper function that converts ratios to del values"""
    ratio = array(ratio)
    return ((ratio / std) - 1) * 1000

def dtor(delta, std):
    """Helper function to convert from del values to a ratio"""
    delta = array(delta)
    return (delta / 1000 + 1) * std

def corr(corrline, r, std):
    """Helper function to corect ratios based on a correction line"""
    r = array(r)
    corrected_ratio = r*corrline[0] + corrline[1]
    return rtod(corrected_ratio, std)

def keysearch(search, data):
    """Function that searches dictionary for particular keys"""
    l = []
    for key in data.keys():
        if search(key):
            l += [key]
    return l

def D17O_excess(d17, d18, factor=0.52):
    """
    Calculate the O-17 excess given d18 and d17 data
    
    inputs
    ------
    d17 : array of floats
    d18 : array of floats
    factor : float
    """
    return d17 - factor * d18

def set_color(s):
    if s in ["USGS34"]:
        return 'magenta'
    elif s in ["USGS35"]:
        return 'red'
    elif s in ["USGS32"]:
        return 'Chartreuse'
    elif s in ["IAEANO3"]:
        return 'yellow'
    else: 
        return 'k'
    
def savestuff(fname, l):
    with open(fname, 'w') as f:
        for line in l:
            s = "\t".join(line)
            f.write(s + "\n")
        f.close()

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


C:\Users\uwisolab\Anaconda2\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


### Standards data
rXX values are resistor values for a given mass XX

In [2]:
irene_standards_date = 'Irene Standards as of: 161121'
SMOW17 = 0.0003799 # The 17O/16O ratio of SMOW, from Li et al., 1988
SMOW18 = 0.0020052 # The 18O/16O ratio of SMOW, from Baertschi 1976
air15 = 0.0036782 # The 15N/14N ratio of air, from De Bievre et al., 1996

IAEANO3 = {'name':'IAEANO3','material':'potassium nitrate','index':[],'d15Nacc':4.72,'d18Oacc':25.32,'d17Oacc':nan,'D17Oacc':-0.18,'notes':'d15N according to Bohlke and Coplen 1993, d18O according to Brand et al. 2009, D17O according to Michalski et al. 2002, d17O calculated from d18O and D17O using 0.52*USGS35["d18Oacc"]+USGS35["D17Oacc]'}
IAEANO3["d17Oacc"] = 0.52*IAEANO3["d18Oacc"]+IAEANO3["D17Oacc"]
USGS35 = {'name':'USGS35','material':'sodium nitrate','index':[],'d15Nacc':2.7,'d18Oacc':56.81,'d17Oacc':nan,'D17Oacc':21.56,'notes':'d15N according to Bohlke et al. 2003, d18O according to Brand et al. 2009, D17O according to Michalski et al. 2002, d17O calculated from d18O and D17O using 0.52*USGS35["d18Oacc"]+USGS35["D17Oacc"]'}
USGS35["d17Oacc"] = 0.52*USGS35["d18Oacc"]+USGS35["D17Oacc"]
USGS35["alpha"] = log(1+USGS35["d17Oacc"]/1000)/log(1+USGS35["d18Oacc"]/1000)
USGS34 = {'name':'USGS34','material':'potassium nitrate','index':[],'d15Nacc':-1.8,'d18Oacc':-27.78,'d17Oacc':nan,'D17Oacc':-0.1,'notes':'d15N according to Bohlke et al. 2003, d18O according to Brand et al. 2009, D17O according to Bohlke et al. 2002'}
USGS34["d17Oacc"] = 0.52*USGS34["d18Oacc"]+USGS34["D17Oacc"]
USGS32 = {'name':'USGS32','material':'potassium nitrate','index':[],'d15Nacc':180,'d18Oacc':25.4,'d17Oacc':nan,'D17Oacc':-0.2,'notes':'d15N acc"]ording to Bohlke and Coplen 1993, d18O acc"]ording to Brand et al. 2009, D17O assumed to be -0.2, d17O calculated from that]'}
USGS32["d17Oacc"] = 0.52*USGS32["d18Oacc"]+USGS32["D17Oacc"]

# Reference gases
d17O_ref_measured = 11.310 # vs. VSMOW
d18O_ref_measured = 22.120 # vs. VSMOW

r17O_ref_measured = dtor(d17O_ref_measured, SMOW17)
r18O_ref_measured = dtor(d18O_ref_measured, SMOW18)

r32 = 300e6
r33 = 300e9
r34 = 100e9
r28 = 300e6
r29 = (300e9**-1 + 30e9 ** -1)**-1


### Load data files

In [15]:
path = "S:/Data/projects/Robinson_WAIS/dxf/170721_WAIS_Robinson_160805/"
files = glob.glob(path + "*dxf")
fname_1 = "170721_WAIS_Robinson_160805" + ".txt"
plotall = False

data = []
for fid in files:
    bites = None
    fullname = re.split(r"[\\|\.]", fid)[-2]
    #print(fullname)
    #print(re.split(r"[__|\-]", fullname))
    run, blah, samp, blah2 = re.split(r"[__|\-]", fullname)
    with open(fid, 'rb') as f:
        bites = f.read()
    odata = get_odata(bites)
    ndata = get_ndata(bites)
    data += [[(samp, run), (odata, ndata)]]
    
d = {key: value for (key, value) in data}
keys = []
for key in d.keys():
    if "USGS35" in key:
        keys += [key]
keys


out = {}
keys = sorted(d.keys(), key=lambda x: int(x[1]))
for key in keys:
    #print(key)
    if plotall:
        fig, ax = subplots()
        fig.canvas.set_window_title(key[1] + ": " + key[0])
        ax.set_xlim(0, 180)
        ax.set_ylim(0, 10000)
    odata = d[key][0]
    
    grad = gradient(odata[:, 1])
    ref_search_start = 40*5
    sample_search_start = 140*5
    refpeak = peakfind(odata[ref_search_start:, 2],  0.5, -5, -0.05, ref_search_start)
    samppeak = peakfind(odata[sample_search_start:, 1],  0.5, -5, -0.05, sample_search_start)
    
    if samppeak == None:
        print("Blank sample: ", key)
        continue
    
    ref_bk_end = refpeak[0] - 35
    ref_bk_start = refpeak[0] - (35 + 20*5)
    
    samp_bk_start = samppeak[0] - (35 + 20*5)
    samp_bk_end = samppeak[0] - 35
    
    
    #print(odata[(samp_bk_start, samp_bk_end), 0])

    t0_ref = odata[ref_bk_start, 0]
    ref_bk_32_fit = polyfit(odata[ref_bk_start:ref_bk_end, 0] - t0_ref, medfilt(odata[ref_bk_start:ref_bk_end, 1], 13), 2)
    ref_bk_33_fit = polyfit(odata[ref_bk_start:ref_bk_end, 0] - t0_ref, medfilt(odata[ref_bk_start:ref_bk_end, 2], 13), 2)
    ref_bk_34_fit = polyfit(odata[ref_bk_start:ref_bk_end, 0] - t0_ref, medfilt(odata[ref_bk_start:ref_bk_end, 3], 13), 2)
    
    t0_samp = odata[samp_bk_start, 0]
    samp_bk_32_fit = polyfit(odata[samp_bk_start:samp_bk_end, 0] - t0_samp, medfilt(odata[samp_bk_start:samp_bk_end, 1], 13), 2)
    samp_bk_33_fit = polyfit(odata[samp_bk_start:samp_bk_end, 0] - t0_samp, medfilt(odata[samp_bk_start:samp_bk_end, 2], 13), 2)
    samp_bk_34_fit = polyfit(odata[samp_bk_start:samp_bk_end, 0] - t0_samp, medfilt(odata[samp_bk_start:samp_bk_end, 3], 13), 2)
    
    ref_bk_fits = [ref_bk_32_fit, ref_bk_33_fit, ref_bk_34_fit]
    samp_bk_fits = [samp_bk_32_fit, samp_bk_33_fit, samp_bk_34_fit]
    
    x0_ref = odata[ref_bk_start:refpeak[1], 0] - t0_ref
    ref_fits = [(fit[0]*x0_ref**2 + fit[1]*x0_ref + fit[2]) for fit in ref_bk_fits]
    x0_samp = odata[samp_bk_start:samppeak[1], 0] - t0_samp
    samp_fits = [(fit[0]*x0_samp**2 + fit[1]*x0_samp + fit[2]) for fit in samp_bk_fits]
    
    if plotall: 

        plot(odata[:, 0], odata[:, 1])
        plot(odata[:, 0], odata[:, 2])
        plot(odata[:, 0], odata[:, 3])

        plot(x0_ref + t0_ref, ref_fits[0])
        plot(x0_ref + t0_ref, ref_fits[1])
        plot(x0_ref + t0_ref, ref_fits[2])

        plot(x0_samp + t0_samp, samp_fits[0])
        plot(x0_samp + t0_samp, samp_fits[1])
        plot(x0_samp + t0_samp, samp_fits[2])

        plot(odata[samppeak, 0], odata[samppeak, 1], 'kv')
        plot(odata[refpeak, 0], odata[refpeak, 1], 'kv')
    
    a32_ref = trapz(odata[refpeak[0]:refpeak[1], 1] - ref_fits[0][refpeak[0] - ref_bk_start], odata[refpeak[0]:refpeak[1], 0])
    a33_ref = trapz(odata[refpeak[0]:refpeak[1], 2] - ref_fits[1][refpeak[0] - ref_bk_start], odata[refpeak[0]:refpeak[1], 0])
    a34_ref = trapz(odata[refpeak[0]:refpeak[1], 3] - ref_fits[2][refpeak[0] - ref_bk_start], odata[refpeak[0]:refpeak[1], 0])
    r17_ref = (a33_ref/r33) / (2*a32_ref/r32 + a33_ref/r33 + a34_ref/r34)
    r18_ref = (a34_ref/r34) / (2*a32_ref/r32 + a33_ref/r33 + a34_ref/r34)
    acorr_33 = r17O_ref_measured / r17_ref
    acorr_34 = r18O_ref_measured / r18_ref
      
    a32 = trapz(odata[samppeak[0]:samppeak[1], 1] - samp_fits[0][samppeak[0] - samp_bk_start], odata[samppeak[0]:samppeak[1], 0])
    a33 = trapz(odata[samppeak[0]:samppeak[1], 2] - samp_fits[1][samppeak[0] - samp_bk_start], odata[samppeak[0]:samppeak[1], 0])
    a34 = trapz(odata[samppeak[0]:samppeak[1], 3] - samp_fits[2][samppeak[0] - samp_bk_start], odata[samppeak[0]:samppeak[1], 0])
    
    #print("raw Reference peak areas: ", a32_ref, a33_ref, a34_ref)
    #print("raw Sample peak areas:", a32, a33, a34)
    
    r17 = acorr_33*(a33/r33) / (2*a32/r32 + a33/r33 + a34/r34)
    r18 = acorr_34*(a34/r34) / (2*a32/r32 + a33/r33 + a34/r34)
    r17_uncorr = (a33/r33) / (2*a32/r32 + a33/r33 + a34/r34)
    r18_uncorr = (a34/r34) / (2*a32/r32 + a33/r33 + a34/r34)
    
    d17O_ref = rtod(r17_ref, SMOW17)
    d18O_ref = rtod(r18_ref, SMOW18)
    D17O_ref = d17O_ref - 0.52*d18O_ref

    d17O = rtod(r17, SMOW17)
    d18O = rtod(r18, SMOW18)
    D17O = d17O - 0.52*d18O
    
    d17O_uc = rtod(r17_uncorr, SMOW17)
    d18O_uc = rtod(r18_uncorr, SMOW18)
    D17O_uc = D17O_excess(d17O_uc, d18O_uc)
    print(key[1], d17O, d18O, D17O, a32)

    
    d_refs = [d17O_ref, d18O_ref, D17O_ref]
    d_samp = [d17O, d18O, D17O]
    out[key] = {"refdata": {"d17O": d17O_ref, "d18O": d18O_ref, "D17O": D17O_ref, "r17O": r17_ref, "r18O": r18_ref}, 
                "sampdata": {"d17O": d17O, "d18O": d18O, "D17O": D17O, "r17O": r17, "r18O": r18},
                "samp_noref": {"d17O" : d17O_uc, "d18O": d18O_uc, "D17O": D17O_uc, "r17O": r17_uncorr, "r18O": r18_uncorr}}

    #print("Reference del values: ", d17O_ref, d18O_ref, D17O_ref)
    #print("Sample raw del values: ", d17O, d18O, D17O)

    #def O_calibrate(data):
data = out
stand_34 = keysearch(lambda x: "USGS34" in x, out)
stand_35 = keysearch(lambda x: "USGS35" in x, out)
stand_IA = keysearch(lambda x: "IAEANO3" in x, out)
stand_32 = keysearch(lambda x: "USGS32" in x, out)

us34_r17 = []
us34_r18 = []
us35_r17 = []
us35_r18 = []
iaea_r17 = []
iaea_r18 = []
us32_r17 = []
us32_r18 = []

for key in stand_34:
    us34_r17 += [data[key]["sampdata"]["r17O"]]
    us34_r18 += [data[key]["sampdata"]["r18O"]]
for key in stand_35:
    us35_r17 += [data[key]["sampdata"]["r17O"]]
    us35_r18 += [data[key]["sampdata"]["r18O"]]    
for key in stand_IA:
    iaea_r17 += [data[key]["sampdata"]["r17O"]]
    iaea_r18 += [data[key]["sampdata"]["r18O"]]
for key in stand_32:
    us32_r17 += [data[key]["sampdata"]["r17O"]]
    us32_r18 += [data[key]["sampdata"]["r18O"]]    
    
    

all_r17 = concatenate((array(us34_r17), array(us35_r17)))
all_r18 = concatenate((array(us34_r18), array(us35_r18)))
us35_D17O = D17O_excess(rtod(array(us35_r17), SMOW17),rtod(array(us35_r18), SMOW18))
us34_D17O = D17O_excess(rtod(array(us34_r17), SMOW17),rtod(array(us34_r18), SMOW18))
all_D17O = concatenate((us34_D17O, us35_D17O))
                                                

u34_r17acc = dtor(USGS34["d17Oacc"], SMOW17)
u34_r18acc = dtor(USGS34["d18Oacc"], SMOW18)

u35_r17acc = dtor(USGS35["d17Oacc"], SMOW17)
u35_r18acc = dtor(USGS35["d18Oacc"], SMOW18)
                        
u35_D17acc = USGS35["D17Oacc"]
u34_D17acc = USGS34["D17Oacc"]

all_acc_r17 = concatenate((ones_like(us34_r17) * u34_r17acc, ones_like(us35_r17) * u35_r17acc))
all_acc_r18 = concatenate((ones_like(us34_r18) * u34_r18acc, ones_like(us35_r18) * u35_r18acc))
all_acc_D17O = concatenate((ones_like(us34_r18) * u34_D17acc, ones_like(us35_r18) * u35_D17acc))

corrline_r17 = linregress(all_r17, all_acc_r17)
corrline_r18 = linregress(all_r18, all_acc_r18)
corrline_D17 = linregress(all_D17O, all_acc_D17O)


x17 = linspace(.99, 1.1, 111) * SMOW17
x18 = linspace(.99, 1.1, 111) * SMOW18
xD17 = linspace(-5, 40, 46)

fig_d17, ax_d17 = subplots()
fig_d18, ax_d18 = subplots()
fig_D17, ax_D17 = subplots()
fig_D17O, ax_D17O = subplots()


ax_d17.plot(rtod(x17, SMOW17), rtod((corrline_r17[0]*x17 +  corrline_r17[1]), SMOW17))
ax_d18.plot(rtod(x18, SMOW18), rtod((corrline_r18[0]*x18 +  corrline_r18[1]), SMOW18))
ax_D17.plot(D17O_excess(rtod(x17, SMOW17),rtod(x18, SMOW18)), D17O_excess(corr(corrline_r17, x17, SMOW17), corr(corrline_r18, x18, SMOW18)))
ax_D17O.plot(xD17, corrline_D17[0]*xD17 + corrline_D17[1])
                        
ll = []
for x in sorted(out.items(), key=lambda x: x[0][1]):
    color = set_color(x[0][0])
    d17 = x[1]["sampdata"]["d17O"]
    d18 = x[1]["sampdata"]["d18O"]
    d17O_corr = corr(corrline_r17, x[1]["sampdata"]["r17O"], SMOW17)
    d18O_corr = corr(corrline_r18, x[1]["sampdata"]["r18O"], SMOW18)
    D17O_corr = D17O_excess(d17O_corr, d18O_corr)
    D17O_corr2 = corrline_D17[0]*D17O_excess(d17, d18) + corrline_D17[1]
    ax_d17.plot(x[1]["sampdata"]["d17O"], d17O_corr, 'o', color=color, label=x[0][0])
    ax_d18.plot(x[1]["sampdata"]["d18O"], d18O_corr, 'o', color=color, label=x[0][0])
    ax_D17.plot(D17O_excess(d17, d18), D17O_corr, 'o',color=color, label=x[0][0])
    ax_D17O.plot(D17O_excess(d17, d18), D17O_corr, 'o',color=color, label=x[0][0])
                        
    ll += [[x[0][0], str(D17O_corr)]]
    #print(x[0], D17O_corr)

#savestuff(fname_1, ll)


C:\Users\uwisolab\Anaconda2\lib\site-packages\ipykernel\__main__.py:44: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


('40790', 12.974584898558295, 25.932385597907004, -0.51025561235334749, 23566.817635824787)
('Blank sample: ', ('Blank', '40791'))
('40792', 54.129585057717698, 68.151625681703194, 18.690739703232033, 19855.864113659234)
('40793', 0.52893337678994179, -0.96488227901836421, 1.0306721618794912, 18928.774651902087)
('40794', 20.829925667206872, 40.781878093246519, -0.37665094128131926, 21368.054715633807)
('40795', 21.234947636853629, 41.818568221596799, -0.51070783837670675, 20498.919194456041)
('40796', -1.1903685976517586, -2.445240966772122, 0.081156705069744817, 19104.357251973659)
('40797', 20.807933490820218, 40.478740198989449, -0.24101141265429504, 20741.450219464368)
('40798', 53.244777067373185, 68.30800528412783, 17.72461431962671, 20627.569548126212)
('40799', 22.600219697973767, 42.551958769317721, 0.47320113792855167, 20585.87146790792)
('40800', 70.570852885651462, 85.458051755214811, 26.132665972939762, 21594.7785242629)
('40801', 70.835219187476156, 84.81683020226005, 26

In [ ]:
ratio = 

In [122]:
print(IAEANO3["D17Oacc"], mean(iaea_D17_corr), std(iaea_D17_corr))
print(USGS34["D17Oacc"], mean(us34_D17_corr), std(us34_D17_corr))
print(USGS32["D17Oacc"], mean(us32_D17_corr), std(us32_D17_corr))
print(USGS35["D17Oacc"], mean(us35_D17_corr), std(us35_D17_corr))

(-0.18, -0.26733902014754785, 0.70829087752824138)
(-0.1, -0.099694457584398322, 0.18854199338358232)
(-0.2, -0.85767031816077244, 0.084979495140161676)
(21.56, 21.559694457584197, 0.28660181635172899)


In [10]:
print(IAEANO3["D17Oacc"], mean(iaea_D17_corr), std(iaea_D17_corr))
print(USGS34["D17Oacc"], mean(us34_D17_corr), std(us34_D17_corr))
print(USGS32["D17Oacc"], mean(us32_D17_corr), std(us32_D17_corr))
print(USGS35["D17Oacc"], mean(us35_D17_corr), std(us35_D17_corr))

NameError: name 'iaea_D17_corr' is not defined

In [123]:
print(IAEANO3["d17Oacc"], mean(iaea_d17_corr), std(iaea_d17_corr))
print(USGS34["d17Oacc"], mean(us34_d17_corr), std(us34_d17_corr))
print(USGS32["d17Oacc"], mean(us32_d17_corr), std(us32_d17_corr))
print(USGS35["d17Oacc"], mean(us35_d17_corr), std(us35_d17_corr))

(12.986400000000001, 12.769365758366403, 0.64338865976121662)
(-14.5456, -14.544578626809855, 0.12037837140815594)
(13.008000000000001, 12.427320229622119, 0.081854025018439899)
(51.101200000000006, 51.100178626809651, 0.23453051485287357)


In [117]:
print(IAEANO3["d17Oacc"], mean(iaea_d17_corr), std(iaea_d17_corr))
print(USGS34["d17Oacc"], mean(us34_d17_corr), std(us34_d17_corr))
print(USGS32["d17Oacc"], mean(us32_d17_corr), std(us32_d17_corr))
print(USGS35["d17Oacc"], mean(us35_d17_corr), std(us35_d17_corr))

(12.986400000000001, 12.442725097070673, 0.33545850975218616)
(-14.5456, -14.543442056403428, 0.22244881054169119)
(13.008000000000001, 12.564008593344795, 0.12198835048321961)
(51.101200000000006, 51.099042056403036, 0.30782179904263735)


In [128]:
print(IAEANO3["d18Oacc"], mean(iaea_d18_corr), std(iaea_d18_corr))
print(USGS34["d18Oacc"], mean(us34_d18_corr), std(us34_d18_corr))
print(USGS32["d18Oacc"], mean(us32_d18_corr), std(us32_d18_corr))
print(USGS35["d18Oacc"], mean(us35_d18_corr), std(us35_d18_corr))

(25.32, 25.070586112526826, 0.33077919954673474)
(-27.78, -27.778623402356651, 0.27773235305499194)
(25.4, 25.548058745736327, 0.087782817183805636)
(56.81, 56.808623402356659, 0.18826677881171563)


In [118]:
print(IAEANO3["d18Oacc"], mean(iaea_d18_corr), std(iaea_d18_corr))
print(USGS34["d18Oacc"], mean(us34_d18_corr), std(us34_d18_corr))
print(USGS32["d18Oacc"], mean(us32_d18_corr), std(us32_d18_corr))
print(USGS35["d18Oacc"], mean(us35_d18_corr), std(us35_d18_corr))

(25.32, 25.087675426405724, 0.32065647540414083)
(-27.78, -27.778676519115557, 0.29794539302549605)
(25.4, 25.533708534965388, 0.12274933819248704)
(56.81, 56.808676519115693, 0.12983878461765577)


In [96]:
for pair in l:
    plot(odata[pair, 0], odata[pair, 1], 'kv', alpha=0.8)

In [272]:
l = []
for row in areas:
    if row[0][0] == "USGS35":
        d17 = rtod((row[1][0][1] / r33) / (2*row[1][0][0] / r32 + row[1][0][1] / r33 + row[1][0][2] / r34), SMOW17)
        d18 = rtod((row[1][0][2] / r34) / (2*row[1][0][0] / r32 + row[1][0][1] / r33 + row[1][0][2] / r34), SMOW18)
        l += [[d17, d18]]
        

In [275]:
[(x[0] - 0.52*x[1]) for x in l]

[33.599074344230907, 32.538711428646842, 35.21273991133895, 33.565312728260018]

In [20]:
fname_1 = "170228" + "_D17O_john_correction.txt"
